## Manipulación Simbólica en Julia

Usaremos el paquete `Symbolics.jl`. 

Es un paquete nuevo y sujeto a cambios drásticos!

In [ ]:
#import Pkg; 
#Pkg.add("Symbolics")
#Pkg.add("Latexify")
using Symbolics
using Latexify
using Plots

Las variables se define con el macro `@variables`

In [ ]:
@variables x y w[1:3]

In [ ]:
typeof(x)

Las *operaciones* no se *hacen* sino que quedan indicadas. Es una forma similar a la que se usa para indicar una expresion, un árbol. 

In [ ]:
z = x^2 + y

In [ ]:
typeof(z)

In [ ]:
z^2

In [ ]:
Symbolics.simplify(z^2; expand=true)

In [ ]:
Symbolics.simplify(z^2)

Podemos trabajar también con matrices o arrays arbitrarios. 

In [ ]:
A = [x^2 + y 0 2x
     0       1 2y
     y^2 + x 0 0]

Se puede mejorar la manera en que las ecuaciones son mostradas con el paquete `Latexify.jl` pero para este caso no hace diferencia (en Jupyter)

In [ ]:
latexify(A)

In [ ]:
using SparseArrays
spA = sparse(A)

Podemos calcular algunas inversas no muy complicadas.

In [ ]:
A_inv = inv(A)

In [ ]:
Idd = A * A_inv

Para encontrar la identidad debemos usar la función `simplify()`, la cual aplicamos a cada elemento con la notación usual de agregar un .

In [ ]:
Symbolics.simplify.(Idd)

También podemos hacer sustituciones. La función `substitute()` admite un diccionario con las sustituciones que queremos hacer.

In [ ]:
r = x^2 + y^2
@variables θ
Symbolics.substitute(r, Dict([x =>sin(θ)]))

In [ ]:
trig = Dict([x => sin(θ), y => cos(θ)])

In [ ]:
Symbolics.substitute(r,trig)

In [ ]:
Symbolics.simplify(Symbolics.substitute(r,trig))

### Derivadas: 

Calculemos un gradiente:

In [ ]:
@variables s[1:3] p[1:3]
ss = Symbolics.scalarize(s)
ps = Symbolics.scalarize(p)

In [ ]:
h(u,t) = u'*u - t'*u

In [ ]:
Symbolics.gradient(h(ss,ps),ss)

Calculemos un Jacobiano:

In [ ]:
function f(u,p)
  [p[1]*u[1] - u[3]; u[1]^2 - u[2]; u[3] + cos(u[2])]
end

#@register_symbolics cos(x) 

In [ ]:
f([1;2;3.],[4;5;6])

In [ ]:
f([x, y, z],p) # Recall that z = x^2 + y

In [ ]:
f(ss,ps)

In [ ]:
Jfs = Symbolics.jacobian(f(ss,ps), ss)

### Generando una función numérica

In [ ]:
Jfs_exp = Symbolics.build_function(Jfs,s, p);
Jfs_f = eval(Jfs_exp[1]);

In [ ]:
Jfs_f([1.;2.;3], [4.;5;6])

**Se pueden generar funciones muy eficientes, por ejemplo construidas con paralelismo incluido.**

### Una aplicación:

Vamos a usar el método de Newton pero de forma symbólica.

In [ ]:
function NR_one_step(f, Jf, x0, par)
    return x0 - Jf(x0,par)\f(x0,par)
end

In [ ]:
function my_sqrt(p,par)
    x0, N = par
    f(x,y) = x^2 - y
    Jf(x,y) = 2x
    x = x0
    for i ∈ 1:N
        x = NR_one_step(f,Jf,x,p)
    end
    return x
end

In [ ]:
par = (1.,3)
my_sqrt(2,par)

In [ ]:
@variables v

In [ ]:
par = (1, 3)
my_sqrt(v,(1,3))

In [ ]:
k_3(v) = Symbolics.simplify(my_sqrt(v,(1,3)), expand=true)
k_3(v)

In [ ]:
k_3_ex = Symbolics.simplify(my_sqrt(v,(1,3)))

Tomamos esta expresión y hacemos una función numérica:

In [ ]:
k_3_exp = Symbolics.build_function(k_3_ex,v)
Base.remove_linenums!(k_3_exp)

In [ ]:
k_3_f = eval(k_3_exp)

In [ ]:
k_3_f(2)

Esta no es sólo una función simbólica sino también numérica.

In [ ]:
#import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools


In [ ]:
@btime k_3_f(2.)

In [ ]:
@btime my_sqrt(2,(1.,3))

In [ ]:
@btime k_3(2.)

In [ ]:
k_3(2) - sqrt(2)

Incluso la podemos plotear y vemos que anda muy bien para valores pequeños de la variable. 

In [ ]:
plt = plot(k_3_f, xlims=(0.01, 30), label="k_3_f", legend=:bottomright)
plot!(plt, sqrt, ls=:dash, label="sqrt", lw=2)

Podemos usar la función original con distintos valores de N para mejorarla. 

In [ ]:
k_5_ex = Symbolics.simplify(my_sqrt(v,(1.,5)))
k_5_exp = Symbolics.build_function(k_5_ex,v)
k_5_f = eval(k_5_exp)

plot!(plt, k_5_f, xlims=(0.01, 30), label="k_5_f")


## NOTA: ##

En muchísimas aplicaciones se necesita calcular Jacobianos de expresiones generales que son muy complicadas o no se conocen de forma previa. Allí es donde esta librería es muy poderosa, lo mismo que otra que se llama **ForwardDiff**

In [ ]:
import Pkg; Pkg.add("ForwardDiff")
using ForwardDiff

In [ ]:
f(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);

x = rand(5) # small size for example's sake

In [ ]:
g = x -> ForwardDiff.gradient(f, x); # g = ∇f

g(x)

In [ ]:
ForwardDiff.hessian(f, x)

### Integración: 

No existe aún. **Pero: podemos usar, por ejemplo, SymPy.jl**

In [ ]:
#import Pkg; Pkg.add("SymPy")
using SymPy

Vamos al notebook SymPy_examples.ipynb

In [ ]:
@vars xs
integrate(xs^2, (xs, 0, 1))